In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dir = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training'
test_dir = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing'

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescaling for the testing set
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 4600 images belonging to 2 classes.
Found 1872 images belonging to 2 classes.


In [4]:
# Load pre-trained VGG-16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
model = Sequential()

# Add VGG-16 base model
model.add(base_model)

# Flatten the output and add a dense layer
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
epochs = 1

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


143/143 [==============================] - 811s 6s/step - loss: 0.5656 - accuracy: 0.8098 - val_loss: 0.1669 - val_accuracy: 0.9370


In [7]:
# Evaluate the model on the test set
score = model.evaluate(test_generator, steps=test_generator.samples // batch_size)

print(f'Test Loss: {score[0]}')
print(f'Test Accuracy: {score[1]}')


58/58 [==============================] - 257s 4s/step - loss: 0.1671 - accuracy: 0.9364
Test Loss: 0.16706128418445587
Test Accuracy: 0.9364224076271057


In [8]:
import tensorflow as tf
from keras.models import load_model

model.save('Brain Tumour.h5')

In [9]:
from keras.models import load_model

In [10]:
from keras import utils

In [11]:
from keras.applications.vgg16 import preprocess_input


In [12]:
import numpy as np

In [13]:
model=load_model('Brain Tumour.h5')

In [14]:
img_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training/Tumour/Cancer (184).jpg'  # Change this to the path of your brain image
img = utils.load_img(img_path, target_size=(224, 224))
img_array = utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [15]:
prediction = model.predict(img_array)
prediction

1/1 [==============================] - 0s 223ms/step


array([[1.]], dtype=float32)

In [16]:
if prediction[0][0] > 0.5:
         print("Tumour detected")
else:
        print("Result is Normal")

Tumour detected
